In [5]:
import os
import pandas as pd
import pickle

def result_table(csv_file_path):
    experiments_folder = "/home/clair01/PycharmProjects/voa-pomdp/experiments_lab/experiments/4blocks"
    
    # Load the CSV data
    df = pd.read_csv(csv_file_path)
    
    # Initialize a list to store the accumulated reward for each row
    accumulated_rewards = []
    
    # Iterate through the rows of the DataFrame
    for index, row in df.iterrows():
        # Extract necessary columns
        belief_idx = row['belief_idx']
        state_idx = row['state_idx']
        help_config_idx_local = row['help_config_idx_local']
        conducted_datetime_stamp = row['conducted_datetime_stamp']
        
        # Define the path to the folder and the .pkl file
        folder_path = os.path.join(experiments_folder, conducted_datetime_stamp)
        pkl_file_path = os.path.join(folder_path, 'results.pkl')  # Assuming the pkl file name
        
        # Check if the .pkl file exists
        if os.path.exists(pkl_file_path):
            # Load the .pkl file
            with open(pkl_file_path, 'rb') as f:
                data = pickle.load(f)
                # Assuming the pkl file contains a dictionary with 'rewards' key
                reward = data.get('rewards', 0)  # Default to 0 if 'rewards' key doesn't exist
                accumulated_rewards.append(sum(reward))
        else:
            # If the pkl file doesn't exist, append 0 or any default value for reward
            accumulated_rewards.append(0)
    
    # Add the 'accumulate_reward' column to the DataFrame
    df['accumulate_reward'] = accumulated_rewards
    
    return df


In [6]:
#build a result table for pouct with 2000 depth policy

# Define file paths
csv_file_path = "/home/clair01/PycharmProjects/voa-pomdp/experiments_lab/configurations/experiments_4_blocks.csv"

experiments_4_blocks_2000_policy_result = result_table(csv_file_path)

experiments_4_blocks_2000_policy_result

,experiment_id,belief_idx,state_idx,help_config_idx_local,belief_mus,belief_sigmas,state,help_config,conducted_datetime_stamp,accumulate_reward
0,1,0,0,-1,"[[-0.6515422291998789, -0.9449885312256132], [...","[[0.13500725243134423, 0.10255128721761153], [...","[[-0.6839691428285296, -0.9870631037995707], [...",NaN,2024-09-04_15-25-34,0.731288
1,2,0,0,0,"[[-0.6515422291998789, -0.9449885312256132], [...","[[0.13500725243134423, 0.10255128721761153], [...","[[-0.6839691428285296, -0.9870631037995707], [...","[-0.5567808869537451, -1.215127556940363, -1.8...",2024-09-04_15-31-02,1.695704
2,3,0,0,1,"[[-0.6515422291998789, -0.9449885312256132], [...","[[0.13500725243134423, 0.10255128721761153], [...","[[-0.6839691428285296, -0.9870631037995707], [...","[0.06857881851979197, 0.21792011149769983, -2....",2024-09-04_15-53-27,3.792802
3,4,0,0,2,"[[-0.6515422291998789, -0.9449885312256132], [...","[[0.13500725243134423, 0.10255128721761153], [...","[[-0.6839691428285296, -0.9870631037995707], [...","[-2.755985913634277, -2.210583901185422, 1.172...",2024-09-04_15-59-19,4.033154
4,5,0,0,3,"[[-0.6515422291998789, -0.9449885312256132], [...","[[0.13500725243134423, 0.10255128721761153], [...","[[-0.6839691428285296, -0.9870631037995707], [...","[0.029355883598327637, -2.341539045373434, -0....",2024-09-04_16-05-14,3.161487
...,...,...,...,...,...,...,...,...,...,...
65,66,1,4,1,"[[-0.8064710911741269, -0.6297700623591925], [...","[[0.11808035964682562, 0.12948951240942116], [...","[[-0.8561366950690926, -0.6624570016103716], [...","[0.06857881851979197, 0.21792011149769983, -2....",2024-09-09_17-41-46,3.716179
66,67,1,4,2,"[[-0.8064710911741269, -0.6297700623591925], [...","[[0.11808035964682562, 0.12948951240942116], [...","[[-0.8561366950690926, -0.6624570016103716], [...","[-2.755985913634277, -2.210583901185422, 1.172...",2024-09-09_17-47-32,1.705498
67,68,1,4,3,"[[-0.8064710911741269, -0.6297700623591925], [...","[[0.11808035964682562, 0.12948951240942116], [...","[[-0.8561366950690926, -0.6624570016103716], [...","[0.029355883598327637, -2.341539045373434, -0....",2024-09-09_18-08-08,2.672359
68,69,1,4,4,"[[-0.8064710911741269, -0.6297700623591925], [...","[[0.11808035964682562, 0.12948951240942116], [...","[[-0.8561366950690926, -0.6624570016103716], [...","[-1.8056453952435314, -1.9710437412927044, -0....",2024-09-09_18-19-43,2.742698


In [7]:
#build a result table for pouct with 20 depth policy

# Define file paths
csv_file_path = "/home/clair01/PycharmProjects/voa-pomdp/experiments_lab/configurations/experiments_4_blocks_shallow_policy.csv"

experiments_4_blocks_shallow_policy_result = result_table(csv_file_path)

experiments_4_blocks_shallow_policy_result


,experiment_id,belief_idx,state_idx,help_config_idx_local,belief_mus,belief_sigmas,state,help_config,conducted_datetime_stamp,accumulate_reward
0,1,0,0,-1,"[[-0.6515422291998789, -0.9449885312256132], [...","[[0.13500725243134423, 0.10255128721761153], [...","[[-0.6839691428285296, -0.9870631037995707], [...",NaN,2024-09-09_17-57-43,0.707139
1,2,0,0,0,"[[-0.6515422291998789, -0.9449885312256132], [...","[[0.13500725243134423, 0.10255128721761153], [...","[[-0.6839691428285296, -0.9870631037995707], [...","[-0.5567808869537451, -1.215127556940363, -1.8...",2024-09-09_18-02-32,2.720402
2,3,0,0,1,"[[-0.6515422291998789, -0.9449885312256132], [...","[[0.13500725243134423, 0.10255128721761153], [...","[[-0.6839691428285296, -0.9870631037995707], [...","[0.06857881851979197, 0.21792011149769983, -2....",2024-09-09_18-42-59,3.723322
3,4,0,0,2,"[[-0.6515422291998789, -0.9449885312256132], [...","[[0.13500725243134423, 0.10255128721761153], [...","[[-0.6839691428285296, -0.9870631037995707], [...","[-2.755985913634277, -2.210583901185422, 1.172...",2024-09-09_18-49-48,3.651204
4,5,0,0,3,"[[-0.6515422291998789, -0.9449885312256132], [...","[[0.13500725243134423, 0.10255128721761153], [...","[[-0.6839691428285296, -0.9870631037995707], [...","[0.029355883598327637, -2.341539045373434, -0....",2024-09-10_11-05-19,2.823836
...,...,...,...,...,...,...,...,...,...,...
65,66,1,4,1,"[[-0.8064710911741269, -0.6297700623591925], [...","[[0.11808035964682562, 0.12948951240942116], [...","[[-0.8561366950690926, -0.6624570016103716], [...","[0.06857881851979197, 0.21792011149769983, -2....",2024-09-12_09-19-16,1.553122
66,67,1,4,2,"[[-0.8064710911741269, -0.6297700623591925], [...","[[0.11808035964682562, 0.12948951240942116], [...","[[-0.8561366950690926, -0.6624570016103716], [...","[-2.755985913634277, -2.210583901185422, 1.172...",2024-09-12_09-24-59,1.624437
67,68,1,4,3,"[[-0.8064710911741269, -0.6297700623591925], [...","[[0.11808035964682562, 0.12948951240942116], [...","[[-0.8561366950690926, -0.6624570016103716], [...","[0.029355883598327637, -2.341539045373434, -0....",2024-09-12_09-30-42,2.666695
68,69,1,4,4,"[[-0.8064710911741269, -0.6297700623591925], [...","[[0.11808035964682562, 0.12948951240942116], [...","[[-0.8561366950690926, -0.6624570016103716], [...","[-1.8056453952435314, -1.9710437412927044, -0....",2024-09-12_09-35-56,1.673578


In [ ]:
#build a result table for handmade policy with 20 depth policy

# Define file paths
csv_file_path = "/home/clair01/PycharmProjects/voa-pomdp/experiments_lab/configurations/experiments_4_blocks_handmade_policy.csv"

eexperiments_4_blocks_handmade_policy_result = result_table(csv_file_path)

eexperiments_4_blocks_handmade_policy_result

In [5]:
from experiments_lab.experiment_results_data import ExperimentResults

res = ExperimentResults.load('experiments/4blocks/2024-09-12_17-16-47/results.pkl')


In [6]:
res.total_reward == sum(res.rewards)

True

1. for each state,belief compute the value difference for each helping action (new table: belief, state, helping-action, value diff)
2. for each belief and helping action, compute the mean value difference over state and also the std
3. plot histogram of the std's (there are n_beliefs x n_helping_actions stds)
    that will let us know how much variance over initial states

4. compute the mean accumulated reward over all experiments for each helping configs (including no help).
    it would be useful to plot 2-4 images of detections from experiments for each  helping config, next to the accumulated reward

5. add to the tables above another column: episode length, and for each helping config compute how many experiments ended in less than 20 steps.
   plot histogram where x-axis is episode length and y-axis is mean accumulated reward + confidence interval
   that would be useful to make sure that episodes that ended early were generally more successful although blocks may fall from the tower

6. bar plot of mean value difference for each helping config, for each policy (3 bars for 3 policies 1 near each other)

7.  bar plot of mean value difference for each helping config, mean over all policies, but different bar for each of the two initial belief
   that will let us know how different were the results for different initial beliefs and let us know if we need to do more experiments for some beliefs
